In [ ]:
# |default_exp mongo

In [ ]:
DB_HOST = "localhost"
DB_PORT = 27017
DB_NAME = "masque_et_la_plume"

In [ ]:
from connection import get_collection

auteurs_coll = get_collection(
    target_db=DB_HOST, client_name=DB_NAME, collection_name="auteurs"
)
logs_coll = get_collection(
    target_db=DB_HOST, client_name=DB_NAME, collection_name="logs"
)
type(auteurs_coll)

pymongo.synchronous.collection.Collection

# Auteur

In [ ]:
# |export

from datetime import datetime
import pymongo.synchronous.collection
from bson import ObjectId


class Auteur:
    def __init__(self, nom: str, collection: pymongo.synchronous.collection.Collection):
        """
        Auteur is a class that represents an author in the database auteurs.
        :param nom: The name of the author.
        :param collection: The collection in which the author is stored.
        """

        self.nom = nom
        self.collection = collection

    def exists(self) -> bool:
        """
        Check if the author exists in the database.
        :return: True if the author exists, False otherwise.
        """

        return self.collection.find_one({"nom": self.nom}) is not None

    def keep(self, logs_coll: pymongo.synchronous.collection.Collection):
        """
        Keep the author in the database.
        """

        if not self.exists():
            logs_coll.insert_one(
                {
                    "operation": "insert",
                    "entite": "auteur",
                    "desc": self.nom,
                    "date": datetime.now(),
                }
            )
            self.collection.insert_one({"nom": self.nom})
        else:
            logs_coll.insert_one(
                {
                    "operation": "update",
                    "entite": "auteur",
                    "desc": self.nom,
                    "date": datetime.now(),
                }
            )

    def remove(self, logs_coll: pymongo.synchronous.collection.Collection):
        """
        Remove the author in the database.
        """
        # remove from database

        self.collection.delete_one({"nom": self.nom})
        logs_coll.insert_one(
            {
                "operation": "delete",
                "entite": "auteur",
                "desc": self.nom,
                "date": datetime.now(),
            }
        )

    def get_oid(self) -> ObjectId:
        """
        Get the object id of the author.
        :return: The object id of the author. (bson.ObjectId)
        None if does not exist.
        """

        document = self.collection.find_one({"nom": self.nom})
        if document:
            return document["_id"]
        else:
            return None

In [ ]:
acteur1 = Auteur("Victor Hugo", auteurs_coll)
acteur1.exists()

False

In [ ]:
acteur1.keep(logs_coll)
acteur1.exists()

True

In [ ]:
acteur1.get_oid()

ObjectId('67792ebd4be9e3777d3292a8')

In [ ]:
acteur1.remove(logs_coll)
acteur1.exists()

False

In [ ]:
acteur1.get_oid()

# extract py

In [ ]:
from nbdev.export import nb_export

nb_export("mongo helper.ipynb", ".")